In [1]:
#### Notizen
# https://mode.com/blog/bridge-the-gap-window-functions
# ttps://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html

## + Pandas + Spark-SQL
# https://medium.com/jbennetcodes/how-to-get-rid-of-loops-and-use-window-functions-in-pandas-or-spark-sql-907f274850e4
# + SQL <-> Pandas: https://medium.com/jbennetcodes/how-to-rewrite-your-sql-queries-in-pandas-and-more-149d341fc53e

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'group':['A','A','A','B','B','B'],
                   'value':[1,2,3,4,5,6]})

mtcars = pd.read_csv('~/Documents/Data/mtcars.csv',index_col='name')
mtcars.head(1)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_type,hp_cumsum
name,,,,,,,,,,,,,
Mazda RX4,21.0,6,160.0,110,3.9,2.62,16.46,0,1,4,4,suv,215


# Group by vs window function

In [3]:
df

,group,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6


## Group by

In [4]:
df.groupby('group').mean()

,value
group,
A,2
B,5


## Window function

In [5]:
# AVG(value) OVER (PARTITION BY group)
# transform() methode für within group aggregation
# => analog zu "OVER(partition by ...)" in SQL:
# => apply get in vielen fällen auch
df1 = df.groupby('group')['value'].apply(np.cumsum)
df2 = df.groupby('group')['value'].transform(np.cumsum)
df3 = df.groupby('group')['value'].cumsum()
print(df1,df2,df3)

0     1
1     3
2     6
3     4
4     9
5    15
Name: value, dtype: int64 0     1
1     3
2     6
3     4
4     9
5    15
Name: value, dtype: int64 0     1
1     3
2     6
3     4
4     9
5    15
Name: value, dtype: int64


..dasselbe..

In [6]:
x = np.arange(1,100)
pd.Series(x).rolling(3,min_periods=1).mean().head()

0    1.0
1    1.5
2    2.0
3    3.0
4    4.0
dtype: float64

# Cumsum

In [7]:
#SELECT sum(hp) OVER (PARTITION BY cyl ORDER BY hp )

In [8]:
mtcars['hp_cumsum'] = (mtcars.sort_values('hp')
                             .groupby('cyl')
                             ['hp']
                             .cumsum());  
mtcars[['cyl','hp','hp_cumsum']].sort_values(['cyl','hp_cumsum']).head(5)

,cyl,hp,hp_cumsum
name,,,
Honda Civic,4,52,52
Merc 240D,4,62,114
Toyota Corolla,4,65,179
Fiat X1-9,4,66,245
Fiat 128,4,66,311


# Rank

In [9]:
# select rank() over (partition by cyl order by hp desc)
mtcars['hp_rank'] = (mtcars.sort_values('hp')
                             .groupby('cyl')['hp']
                             .rank(method='first', ascending=False));  
mtcars[['cyl','hp','hp_rank']].sort_values(['cyl','hp_rank']).head(5)

,cyl,hp,hp_rank
name,,,
Lotus Europa,4,113,1.0
Volvo 142E,4,109,2.0
Toyota Corona,4,97,3.0
Merc 230,4,95,4.0
Datsun 710,4,93,5.0


In [10]:
# SELECT sum(hp) OVER (PARTITION BY cyl ORDER BY hp rows between 2 preceding and current)

mtcars['hp_cumsum'] = (mtcars.sort_values('hp')
                             .groupby('cyl')
                             ['hp']
                             .apply(lambda x : x.rolling(3).sum()));  
mtcars[['cyl','hp','hp_cumsum']].sort_values(['cyl','hp_cumsum']).head(5)

,cyl,hp,hp_cumsum
name,,,
Toyota Corolla,4,65,179.0
Fiat X1-9,4,66,193.0
Fiat 128,4,66,197.0
Porsche 914-2,4,91,223.0
Datsun 710,4,93,250.0


In [11]:
# Differenz zwischen den Werten
mtcars['diff_hp_in_cyl'] = (mtcars
                            .sort_values('hp')
                            .groupby('cyl')
                            ['hp']
                            .apply(
                                lambda x : x - x.shift())
                            .fillna(-1))

# mean für jede gruppe
(mtcars.sort_values('hp')
    .groupby('cyl')
    ['hp'].apply(lambda x : np.mean((x - x.shift()))))



cyl
4     6.100000
6    11.666667
8    14.230769
Name: hp, dtype: float64

In [12]:
# Komplexe Transformationen von Gruppen
def top_n(df,n=5):
    df['hp_cyl_mean'] = df.hp.mean()
    return df.head(n)

mtcars.groupby('cyl',group_keys=False).apply(top_n,2)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_type,hp_cumsum,hp_rank,diff_hp_in_cyl,hp_cyl_mean
name,,,,,,,,,,,,,,,,
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,suv,250.0,5.0,2.0,82.636364
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,suv,NaN,10.0,10.0,82.636364
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,suv,NaN,4.0,5.0,122.285714
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,suv,325.0,5.0,0.0,122.285714
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,smart,500.0,12.0,0.0,209.214286
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,smart,720.0,4.0,0.0,209.214286


In [13]:
mtcars.groupby('cyl').apply(np.mean) # => fasst zusammen
mtcars.groupby('cyl').apply(lambda x: x.rank(ascending=False)) # hier nicht

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_type,hp_cumsum,hp_rank,diff_hp_in_cyl
name,,,,,,,,,,,,,,,
Mazda RX4,2.5,4.0,5.5,5.0,3.5,7.0,6.0,6.0,2.0,3.5,3.5,2.0,NaN,4.0,3.0
Mazda RX4 Wag,2.5,4.0,5.5,5.0,3.5,5.0,5.0,6.0,2.0,3.5,3.5,2.0,5.0,3.0,5.0
Datsun 710,8.5,6.0,6.0,5.0,8.0,5.0,7.0,5.5,4.5,6.5,9.0,6.0,5.0,7.0,7.0
Hornet 4 Drive,1.0,4.0,1.0,5.0,6.0,4.0,2.0,2.5,5.5,6.5,6.5,2.0,4.0,2.0,5.0
Hornet Sportabout,2.0,7.5,5.5,11.5,6.5,12.0,9.0,7.5,8.5,8.5,12.5,7.5,11.0,3.0,11.0
Valiant,6.0,4.0,2.0,7.0,7.0,1.0,1.0,2.5,5.5,6.5,6.5,5.5,NaN,1.0,7.0
Duster 360,11.0,7.5,5.5,3.5,5.0,9.5,11.0,7.5,8.5,8.5,4.5,7.5,3.0,11.0,11.0
Merc 240D,7.0,6.0,1.0,10.0,11.0,1.0,3.0,5.5,10.0,6.5,3.5,6.0,NaN,2.0,3.0
Merc 230,8.5,6.0,2.0,4.0,7.0,2.0,1.0,5.5,10.0,6.5,3.5,6.0,4.0,8.0,7.0


In [14]:
mtcars.groupby('cyl').transform(np.mean)

,mpg,disp,hp,drat,wt,qsec,vs,am,gear,carb,hp_cumsum,hp_rank,diff_hp_in_cyl
name,,,,,,,,,,,,,
Mazda RX4,19.742857,183.314286,122.285714,3.585714,3.117143,17.977143,0.571429,0.428571,3.857143,3.428571,355.000000,4.0,9.857143
Mazda RX4 Wag,19.742857,183.314286,122.285714,3.585714,3.117143,17.977143,0.571429,0.428571,3.857143,3.428571,355.000000,4.0,9.857143
Datsun 710,26.663636,105.136364,82.636364,4.070909,2.285727,19.137273,0.909091,0.727273,4.090909,1.545455,247.333333,6.0,5.454545
Hornet 4 Drive,19.742857,183.314286,122.285714,3.585714,3.117143,17.977143,0.571429,0.428571,3.857143,3.428571,355.000000,4.0,9.857143
Hornet Sportabout,15.100000,353.100000,209.214286,3.229286,3.999214,16.772143,0.000000,0.142857,3.285714,3.500000,616.916667,7.5,13.142857
Valiant,19.742857,183.314286,122.285714,3.585714,3.117143,17.977143,0.571429,0.428571,3.857143,3.428571,355.000000,4.0,9.857143
Duster 360,15.100000,353.100000,209.214286,3.229286,3.999214,16.772143,0.000000,0.142857,3.285714,3.500000,616.916667,7.5,13.142857
Merc 240D,26.663636,105.136364,82.636364,4.070909,2.285727,19.137273,0.909091,0.727273,4.090909,1.545455,247.333333,6.0,5.454545
Merc 230,26.663636,105.136364,82.636364,4.070909,2.285727,19.137273,0.909091,0.727273,4.090909,1.545455,247.333333,6.0,5.454545


In [15]:
time_key = pd.TimeGrouper('5min')
# ...  .groupby(['key', time_key]).sum())

AttributeError: module 'pandas' has no attribute 'TimeGrouper'